# Apresentação ✒️

Notebook destinado ao estudo da implementação de banco de dados vetoriais, utilizando o LangChain. Os banco de dados vetoriais (vector databases) referem-se a um tipo de banco de dados construídos por termos vetoriais.

A sua particular vantagem quando trabalhados no contexto das LLM's é que conseguem armazenar informação em sua representação vetorial, o que facilita a consulta futura de seus elementos constituintes com base na proximidade do elemento de consulta com o que está presente no banco de dados.

O seu uso pode se associar a implementação de aplicações de LLM's com a técnica de RAG, consulta em documentos ou servir de elemento de memória para um chatbot.

O estudo presente se relaciona a um caso de uso referente a um dataset formato csv. Nele, existem uma determinada quantidade de itens de vestuário, os quais o usuário pode querer consultar em ordem de saber alguma informação. Portanto, concebe-se que se refere a um caso de consulta do tipo Q&A.

O objetivo é transformar os elementos presentes nesse conjunto de dados para uma representação vetorial, que será armazena num espaço vetorial, de modo a facilitar o encontro de determinados elementos com base na consulta do usuário, fornecendo a resposta desejada com base em sua pergunta ou entrada.

No primeiro momento, desenvolve-se uma implementação manual, para destrinchar os passos importantes, e depois uma implementação mais automatizada, utilizando o LangChain.


# Biblioteca 📚

In [1]:
!pip install --upgrade langchain -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 3.2 MB/s eta 0:00:00


In [2]:
!pip install google-generativeai -q

In [6]:
!pip install langchain_google_genai -q

In [13]:
!pip install docarray -q

In [16]:

import warnings
warnings.filterwarnings('ignore')

import os

import google.generativeai as genai

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.embeddings import GooglePalmEmbeddings
from langchain.indexes import VectorstoreIndexCreator

## Dataset 💾

url = https://github.com/Ryota-Kawamura/LangChain-for-LLM-Application-Development/blob/main/OutdoorClothingCatalog_1000.csv

In [10]:
# Carregando o dataset

file = '/content/OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)

In [22]:
# Armazenando o arquivo numa variável

docs = loader.load()

In [25]:
# Visualizando um elemento do arquivo. Nota-se que refere-se a um item de vestuário,
# contando com a sua página, nome, descrição, medidas e etc.

docs[0]

Document(page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.", metadata={'source': '/content/OutdoorClothingCatalog_1000.csv', 'row': 0})

In [27]:
# Para fis de estudo, irei extrair apenas 50 itens e não 1000, para facilitar
# o processamento de vetorização do modelo de embedding.

docs = docs[:50]

## Variáveis do ambiente

In [18]:
os.environ['GOOGLE_API_KEY'] = 'sua-api-key'

genai.configure(api_key=os.environ['GOOGLE_API_KEY'])

## Definindo o modelo de LLM e de embedding

---



In [19]:
# Instanciando o modelo de LLM de uso :

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro-latest",
    temperature=0.2)

In [21]:
# Instanciando o modelo de embedding utilizado :

"""
Importante lembrar que o modelo de embedding é aquele responsável
por transformar o texto disposto em linguagem natural numa representação
numérica vetorial que permite a máquina identificar relações entre os termos.
"""

embedding = GooglePalmEmbeddings()

## Realizando a consulta de forma manual

### Criando um Vector Database

In [28]:
# Criando um banco de dados vetorial (vector database), por meio da combinação
# dos elementos presentes no dataset, que foram armazenados em 'docs', bem como
# com o modelo de embedding.

"""
Aqui estou criando um objeto database, o qual por meio da classe
DocArrayInMemorySearch relaciona os dados (que aqui estão no formato csv)
com o modelo de embedding de uso. Assim, combina-os ao modelo, vetorizando-os
para depois armazená-los num espaço vetorial.
"""

database = DocArrayInMemorySearch.from_documents(
    docs,
    embedding)

In [29]:
# Objeto vector database

database

In [30]:
# Realizando a consulta :

query = "Please suggest a shirt with sunblocking"

In [38]:
q_docs = database.similarity_search(query)

In [32]:
# Visualizando a quantidade de itens que sejam próximos
# do que fora perguntado :

len(q_docs)

4

In [33]:
# Visualizando o terceiro elemento :

q_docs[2]

Document(page_content=": 14\nname: Easy-Care Classic Stripe Short-Sleeve Shirt, Traditional Fit\ndescription: The ultimate striped chambray shirt – we kept everything you love and made it even better. Light and breathable in 100% cotton, with an easy-care finish that sheds wrinkles remarkably well. \n\nSize & Fit\nTraditional Fit: Relaxed through the chest, sleeve and waist. \n\nWhy We Love It\nEven when something is loved as much as a classic chambray, there's always something you can do to make it even better. And that's just what we did – introducing our exclusive easy-care chambray. It's light, breathable and sheds wrinkles. Just throw it in the wash, and it's ready to wear right out of the dryer. \n\nFabric & Care\n100% cotton chambray. A light iron gives it an even crisper look. Machine wash and dry. \n\nAdditional Features\nSingle patch pocket. Button-down collar. Imported. \n\nQuestions?\nFeel free to contact us with any questions or inquiries.", metadata={'source': '/content/O

In [39]:
q_docs = "".join([q_docs[i].page_content for i in range(len(q_docs))])


In [40]:
response = llm.call_as_llm(f"{q_docs} Question: Please list all your \
shirts with sun protection in a table in markdown and summarize each one.")


In [41]:
# Visualizando a resposta da consulta.

display(Markdown(response))

Here are the shirts with sun protection:

| Shirt Name | Sun Protection | Summary |
|---|---|---|
| Performance Plus Woven Shirt | UPF 40+ | Breathable, quick-drying shirt perfect for trail or travel. Made with abrasion-resistant fabric that blocks 98% of harmful UV rays. |
| Refresh Swimwear, V-Neck Tankini Contrasts | UPF 50+ |  A comfortable and stylish tankini top designed for swimming and watersports. Made with recycled nylon and Lycra® spandex, it provides the highest rated sun protection. | 


In [34]:
# Criando um objeto que recupera a informação do banco de dados,
# afinal retriever significa recuperador.

retriever = database.as_retriever()

## Realizando a consulta com o LangChain

In [42]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    verbose=True
)

In [43]:
query =  "Please list all your shirts with sun protection in a table \
in markdown and summarize each one."

In [44]:
response = qa_stuff.run(query)



> Entering new RetrievalQA chain...

> Finished chain.


In [45]:
display(Markdown(response))

| Shirt Name | Summary |
|---|---|
| Performance Plus Woven Shirt | A breathable, quick-drying shirt perfect for trail or travel. It offers UPF 40+ sun protection. |
| Refresh Swimwear, V-Neck Tankini Contrasts | A stylish and sustainable tankini top designed for swimming and watersports. It provides UPF 50+ sun protection. |
| Angler's Athletic Shorts | While not a shirt, these athletic shorts designed for fly-fishing offer UPF 50+ sun protection. | 


### Chain Type ❗

Observando novamente os parâmetros passados pelo RetrievalQA do LangChain, nota-se um parâmetro importante, o `chain_type`. Ele se refere ao tipo de cadeia criada. Ela é importante, pois informa como a informação recuperada do banco de dados será enviada ao modelo de LLM. Há três tipos de cadeias : stuff, map_reduce e refine.

No presente notebook vemos a cadeia stuff. A sua dinâmica se relaciona ao fato de que essa cadeia retorna à LLM toda a informação recuperada pelo método retrieval, sem haver ponderações de nenhuma porção. Desse modo, uma vez utilizada, a LLM conta com todos os dados recuperados para formação de sua resposta, com base no prompt do usuário.

A cadeia map_reduce é diferente da stuff, pois ao invés de fornecer à LLM toda a informação recuperada, prove apenas uma parte reduzida dela, previamente analisada. Assim, a cada porção a LLM vai recebendo a informação, construindo por meio da integração dessas partes uma resposta final e coesa. Essa abordagem é útil para casos de otimização no qual há uma grande quantidade de dados no dataset.

Por fim, a cadeia refine apresenta uma abordagem de refinamento, na forma que é apresentado a ela um documento por vez, produzindo uma resposta parcial. A cada vez que lhe é apresentado um documento, elabora uma resposta parcial, de modo as integrá-las no fim numa resposta final coesa. Seu uso pode ser oportuno em cenários exploratórios, nos quais a resposta pode evoluir à medida que mais informações são consideradas.